# Style Checker WebSocket Demo

This notebook demonstrates how to connect to the Style Guard WebSocket service and send text for style checking.

In [11]:
import json
import websockets
import asyncio
from IPython.display import display, Markdown

## Define WebSocket Connection Function

In [16]:
async def connect_to_style_guard(style_prompt, text_samples):
    uri = "ws://localhost:8000/api/style-guard"
    
    async with websockets.connect(uri) as websocket:
        # Send initial style prompt
        await websocket.send(json.dumps({"style_prompt": style_prompt}))
        
        # Process each text sample
        for i, text in enumerate(text_samples):
            print(f"\n--- Sending Text Sample {i+1} ---")
            print(f"Text: {text[:100]}..." if len(text) > 100 else f"Text: {text}")
            
            # Send the text for checking
            await websocket.send(json.dumps({"text": text}))
            
            # Continue receiving messages until we get the "done" signal
            while True:
                response = await websocket.recv()
                response_data = json.loads(response)
                
                if response_data.get("status") == "done":
                    break
                    
                print("\nResponse Received:")
                if "text_with_comments" in response_data:
                    display(Markdown(f"**Text with Comments:**\n{response_data['text_with_comments']}"))
                else:
                    print(response_data)
            
            # Small delay between requests
            await asyncio.sleep(1)

## Define Sample Text and Style Guide

In [18]:
# Define a style guide prompt
style_guide = """
1. Use active voice whenever possible.
2. Keep sentences short and concise, ideally under 20 words.
3. Avoid jargon and technical terms without explanation.
4. Use Oxford comma in lists.
5. Maintain a professional but conversational tone.
6. Use 'we' instead of 'I' for company communications.
7. Numbers less than 10 should be spelled out.
"""

# Define some text samples for style checking
text_samples = [
    # "The product was developed by our team over a period of six months and it has been lauded by critics for its innovative features, user-friendly interface, and cutting-edge technology.",
    
    # "I believe that the implementation of the new system will significantly improve productivity, efficiency and customer satisfaction.",
    
    "The data was analyzed by the algorithm and 5 key insights were identified.",
    
    "We are excited to announce the launch of our new product that will revolutionize the way you work, play, and interact with technology on a daily basis, making your life easier and more productive than ever before."
]

## Run the WebSocket Connection

In [19]:
# Run the websocket connection
try:
    await connect_to_style_guard(style_guide, text_samples)
except Exception as e:
    print(f"Error connecting to websocket: {e}")


--- Sending Text Sample 1 ---
Text: The data was analyzed by the algorithm and 5 key insights were identified.

Response Received:
{'status': 'style_prompt_updated'}

Response Received:
{'original_text': 'The data was analyzed by the algorithm and 5 key insights were identified.', 'comment': 'Use active voice and spell out numbers under 10.', 'suggestion': 'The algorithm analyzed the data and identified five key insights.'}

--- Sending Text Sample 2 ---
Text: We are excited to announce the launch of our new product that will revolutionize the way you work, p...

Response Received:
{'original_text': 'We are excited to announce the launch of our new product that will revolutionize the way you work, play, and interact with technology on a daily basis, making your life easier and more productive than ever before.', 'comment': 'The sentence is too long, with 39 words. Keep sentences under 20 words for conciseness.', 'suggestion': 'We are excited to announce the launch of our new product. 

## Alternative Approach with Manual Await (if needed)

In [ ]:
# If the await above doesn't work directly in Jupyter, use this approach
# asyncio.run(connect_to_style_guard(style_guide, text_samples))

## Update Style Guide Example

In [ ]:
# Define a different style guide
new_style_guide = """
1. Use a casual, friendly tone throughout.
2. Incorporate first-person perspective ("I" statements).
3. Keep paragraphs short - maximum 3 sentences.
4. Use contractions (don't, can't, won't) freely.
5. Include rhetorical questions to engage readers.
6. Avoid complex vocabulary when simpler alternatives exist.
"""

# New text samples
new_text_samples = [
    "The corporation endeavors to maximize shareholder value through the implementation of strategic organizational initiatives.",
    
    "Wouldn't you like to know more about our amazing new features? I think you'll find them super helpful for your daily tasks!"
]

# Run with new style guide
# Uncomment to run
# await connect_to_style_guard(new_style_guide, new_text_samples)